In [ ]:
# GBM boosting

import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/content/loan_data.csv")


In [ ]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [ ]:
#data procesing

df.dropna(inplace = True)
df.drop(columns=["Loan_ID"], inplace = True)
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [ ]:
from pandas.core.dtypes.api import is_categorical
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
# lấy các cột dạng chữ
is_category = df.dtypes == object
is_category

Gender                True
Married               True
Dependents            True
Education             True
Self_Employed         True
ApplicantIncome      False
CoapplicantIncome    False
LoanAmount           False
Loan_Amount_Term     False
Credit_History       False
Property_Area         True
Loan_Status           True
dtype: bool

In [ ]:
category_column_list = df.columns[is_category].tolist()
category_column_list

['Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [ ]:
df[category_column_list] = df[category_column_list].apply(lambda col:le.fit_transform(col))
df


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1
5,1,1,2,0,1,5417,4196.0,267.0,360.0,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,2900,0.0,71.0,360.0,1.0,0,1
610,1,1,3,0,0,4106,0.0,40.0,180.0,1.0,0,1
611,1,1,1,0,0,8072,240.0,253.0,360.0,1.0,2,1
612,1,1,2,0,0,7583,0.0,187.0,360.0,1.0,2,1


In [ ]:
#chia dữ liệu train test 
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)
x_train = train.drop(columns = ["Loan_Status"])
y_train = train["Loan_Status"]

x_test = test.drop(columns = ["Loan_Status"])
y_test = test["Loan_Status"]

In [ ]:
x_train

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
172,1,1,3,1,0,3522,0.0,81.0,180.0,1.0,0
288,0,0,0,0,0,4124,0.0,115.0,360.0,1.0,1
607,1,1,2,1,0,3987,1411.0,157.0,360.0,1.0,0
55,1,1,2,0,0,2708,1167.0,97.0,360.0,1.0,1
461,1,1,3,0,0,7740,0.0,128.0,180.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...
141,1,0,0,0,0,5417,0.0,168.0,360.0,1.0,2
350,1,1,0,0,0,9083,0.0,228.0,360.0,1.0,1
445,1,1,1,0,1,3466,1210.0,130.0,360.0,1.0,0
558,1,1,0,0,0,3887,2669.0,162.0,360.0,1.0,1


In [ ]:
#train model

from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(learning_rate=0.01,random_state=42)
model.fit(x_train, y_train)



GradientBoostingClassifier(learning_rate=0.01, random_state=42)

In [ ]:
# tinh acc

acc = model.score(x_test, y_test)
print(acc)

0.8020833333333334


In [ ]:
# xg boost
import xgboost as xgb
model_xgb = xgb.XGBClassifier(random_state = 42)
model_xgb.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [ ]:
acc = model_xgb.score(x_test, y_test)
print(acc)

0.8125


In [ ]:
from sklearn.metrics import classification_report
y_pred = model_xgb.predict(x_test)
# In bao cao ket qua
print(classification_report( y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.61      0.65        28
           1       0.85      0.90      0.87        68

    accuracy                           0.81        96
   macro avg       0.78      0.75      0.76        96
weighted avg       0.81      0.81      0.81        96

